<a href="https://colab.research.google.com/github/NbtKmy/gc_workshops/blob/main/NutzlicherAppMitChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nutzlicher App mit ChatGPT?

zum Beispiel - die neu publizierten Aufsätze in ArXiv holen und mit ChatGPT zusammenfassen

## Zu ArXiv API
https://info.arxiv.org/help/api/index.html

Für Python gibt es eine Library "[arxiv](https://pypi.org/project/arxiv/)". Damit kann man leicht den ArXiv API bedienen. 

Als erster Schritt probieren wir ein einfache Query aus:


In [ ]:
!pip install arxiv

In [ ]:
import arxiv

search = arxiv.Search(
    # Formulierung der Query siehe: https://info.arxiv.org/help/api/user-manual.html#query_details
    query = "cat:cs.AI",
    max_results = 5,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    sort_order=arxiv.SortOrder.Descending, 
)



for result in search.results():
    # Das Ergebnis kann man in Form von "result.title" holen. Siehe die Erläuterung von arxiv Library unter "result" https://pypi.org/project/arxiv/
    titleAndSum = result.title + "\n" + result.summary + "\n\n"
    print(titleAndSum)

# Summary noch kürzer machen mit ChatGPT



In [ ]:
!pip install openai

In [ ]:
import openai
import arxiv

my_api_key = ""#@param{type:'string'} 
openai.api_key = my_api_key

def sum_Summary(result):
    system = """Bitte den Aufsatz immer nur in 3 Punkten auf Deutsch kurz zusammenfassen. Die Zusammenfassung sieht wie folgt aus:```
    Titel auf Deutsch:
    - Punkt1
    - Punkt2
    - Punkt3
    ```"""

    text = f"title: {result.title}\nbody: {result.summary}"
    response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[
                    {'role': 'system', 'content': system},
                    {'role': 'user', 'content': text}
                ],
                temperature=0.25,
            )
    summary = response['choices'][0]['message']['content']

    return summary

search = arxiv.Search(
    # Formulierung der Query siehe: https://info.arxiv.org/help/api/user-manual.html#query_details
    query = "cat:cs.AI",
    max_results = 1,
    sort_by=arxiv.SortCriterion.SubmittedDate,
    sort_order=arxiv.SortOrder.Descending, 
)

newsBody = ""
for result in search.results():
    sum = sum_Summary(result)
    authoren = ""
    for autor in result.authors:
        authoren += f" {autor.name},"
    artInfo = f"Artikeltitel: {result.title}\nKurze Zusammenfassung:\n{sum}\nAutor(en):{authoren}\nPubliziert: {result.published}\nLink: {result.entry_id}\n-------------"
    print(artInfo)
    newsBody += artInfo


Artikeltitel: DualCross: Cross-Modality Cross-Domain Adaptation for Monocular BEV Perception
Kurze Zusammenfassung:
Titel auf Deutsch: DualCross: Cross-Modalitäts- und Cross-Domain-Anpassung für monokulare BEV-Wahrnehmung
- Die Schließung der Lücke zwischen Training und Einsatz sowie die Integration mehrerer Sensor-Modalitäten sind zwei schwierige, aber entscheidende Themen für selbstfahrende Fahrzeuge.
- Bestehende Arbeiten konzentrieren sich nur auf eines der oben genannten Themen und übersehen dabei die gleichzeitige Domain- und Modalitätsverschiebung, die in realen Szenarien allgegenwärtig ist.
- In diesem Artikel wird DualCross vorgestellt, ein Framework für Cross-Modalitäts- und Cross-Domain-Anpassung, um das Lernen eines robusteren monokularen Bird's-Eye-View (BEV)-Wahrnehmungsmodells zu erleichtern.
Autor(en): Yunze Man, Liang-Yan Gui, Yu-Xiong Wang,
Publiziert: 2023-05-05 17:58:45+00:00
Link: http://arxiv.org/abs/2305.03724v1
-------------


# Mail senden mit smtplib

Die Library smtplib gehört zu Standardlibrary von Python. Man muss sie nicht extra installieren. 
Mit dieser Library kann man zum Beispiel eine Mail von dem eigenen Google-Mail-Account senden. 
Dafür braucht man das App-Passwort des Google-Mail-Account zu erstellen. 



In [ ]:
import smtplib
from email.mime.text import MIMEText
 

from_email ="****@gmail.com" #@param{type:'string'} 
to_email = "***@***" #@param{type:'string'} 
mail_title = "AI news"
message = newsBody

msg = MIMEText(message, "plain")
msg["Subject"] = mail_title
msg["To"] = to_email
msg["From"] = from_email

 
smtp_host = "smtp.gmail.com"
smtp_port = 587
smtp_password = "App-Passwort" #@param{type:'string'} 
 
server = smtplib.SMTP(smtp_host, smtp_port)
server.starttls()
server.login(from_email, smtp_password)
server.send_message(msg)
server.quit()

## Mitteilung an MSTeams

voraussetzung: "Incoming Webhook" Connector in einem Team erstellen und einen Link dafür haben


In [ ]:
!pip install pymsteams

In [ ]:

import pymsteams

incoming_webhook_url = ""#@param{type:'string'}

teams_obj = pymsteams.connectorcard(incoming_webhook_url)
teams_obj.title("Neuigkeit")
teams_obj.text(newsBody)
teams_obj.send()